In [ ]:
"""
    nn.Linear()
    

    creating model : 
        className(nn.module): 
        def __init__(self, args, *kargs): 
            super(ClassName, self).__init__():
            
            #define what you want...
            ...
"""

In [2]:
import torch 
import torch.nn as nn 

import sklearn.datasets as datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import numpy as np 

# prepare the data 
bc = datasets.load_breast_cancer()


x, y = bc.data, bc.target

n_samples, n_features = x.shape
print(f'n_samples : {n_samples}, n_features : {n_features}')


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1234)

# sclae the features 
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

x_train = torch.from_numpy(x_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
x_test = torch.from_numpy(x_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))


y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)





class LogisticRegression(nn.Module): 

    def __init__(self, n_input_features): 
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        

    def forward(self, x): 
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

   
    
model = LogisticRegression(n_features)

# loss and optimizer 
laerning_rate = 0.01
criterion = nn.BCELoss() # Binary cross entropy loss 
optimizer = torch.optim.SGD(
    model.parameters(), 
    lr=laerning_rate
)


epochs = 100

for epoch in range(epochs): 

    # forward pass
    y_pred = model(x_train)
    # loss 
    loss = criterion(y_pred, y_train)
    # backward pass (gradiet)
    loss.backward()
    # update the params 
    optimizer.step()
    # empty gradient for next epoch 
    optimizer.zero_grad()



    # printing metrics and evaluation
    if (epoch+1) % 10 == 0: 
        msg = f'epoch {epoch} \t loss:{loss.item():.4f} \t ' 
    # evaluation of the model 
    with torch.no_grad(): 
        y_pred = model(x_test)
        # y_predicted_cls = (y_pred > 0.5)
        y_predicted_cls = y_pred.round()

        acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
        print(msg +f'accuracy = {acc:.4f}')        




n_samples : 569, n_features : 30
epoch 9 	 loss:0.5822 	 accuracy = 0.7632
epoch 19 	 loss:0.4947 	 accuracy = 0.8070
epoch 29 	 loss:0.4346 	 accuracy = 0.8509
epoch 39 	 loss:0.3906 	 accuracy = 0.8596
epoch 49 	 loss:0.3570 	 accuracy = 0.8684
epoch 59 	 loss:0.3303 	 accuracy = 0.8684
epoch 69 	 loss:0.3085 	 accuracy = 0.8772
epoch 79 	 loss:0.2904 	 accuracy = 0.8772
epoch 89 	 loss:0.2749 	 accuracy = 0.8772
epoch 99 	 loss:0.2616 	 accuracy = 0.8772


>  #### :warning: whenever you want to manipulate a function that actually perfoms operation with the weighs, and you actually don't need it in the backward loss then you just do with torch.no_gra(): and call it

In [4]:
model(x_test[:5]).detach().numpy() > 0.5

array([[ True],
       [ True],
       [False],
       [ True],
       [ True]])